In [ ]:
from keras.datasets import mnist
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import to_categorical
import numpy as np
import cv2
import os

In [ ]:
actions = ["BBAS3.SA", "CSNA3.SA", "PETR4.SA", "VALE3.SA"]

def loadFiles():
  buy_action_images = []
  sell_action_images = []

  for action in actions:
    buy_path = "./" + action + "./imagens/treino/comprar"
    sell_path = "./" + action + "./imagens/treino/vender"

    for r, _, f in os.walk(buy_path):
      for file in f:
        filepath = os.path.join(r, file)
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

        img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_CUBIC)
        buy_action_images.append(img)

    for r, _, f in os.walk(sell_path):
      for file in f:
        filepath = os.path.join(r, file)
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

        img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_CUBIC)
        sell_action_images.append(img)

  return np.array(buy_action_images), np.array(sell_action_images)
  
buy_action_images, sell_action_images = loadFiles()

In [ ]:
plt.figure(0)
plt.subplot(221)
plt.imshow(buy_action_images[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(buy_action_images[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(buy_action_images[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(buy_action_images[3], cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
import numpy as np
print(buy_action_images.shape)
print(sell_action_images.shape)
x_train = np.concatenate((buy_action_images, sell_action_images), axis=0)
y_train = np.concatenate((np.ones(len(buy_action_images)), np.zeros(len(sell_action_images))), axis=0)
print(x_train.shape)
print(y_train.shape)

In [ ]:
img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
print(x_train.shape)
input_shape = (img_rows, img_cols, 1)
x_train = x_train / 255

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
print(encoded_Y)

In [ ]:
# Cria o modelo
model = Sequential()
#Convolução 2D com função de ativação Rectified Linear Units 32 kernels/Pesos (filtros)

model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))#
#model.add(Conv2D(10, (3, 3), input_shape=(64, 64, 3), activation='relu'))#
#model.add(Conv2D(10, (3, 3), activation='relu'))#

print( model.output_shape)
#Camada de Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
print( model.output_shape)
#Convolução 2D com função de ativação Rectified Linear Units 64 kernels/Pesos (filtros)
model.add(Conv2D(64, (5, 5), activation='relu'))
#model.add(Conv2D(10, (3, 3), activation='relu'))#
#model.add(Conv2D(10, (3, 3), activation='relu'))#
print( model.output_shape)
#Camada de Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
print( model.output_shape)
#Remove 20% das ativações de entrada aleatoriamente
model.add(Dropout(0.2))
#Converte o conjunto de imagens e um vetor unidimensional para a entrada da rede neural totalmente conectada
model.add(Flatten())
print( model.output_shape)

model.add(Dense(32, activation='relu'))
print( model.output_shape)
model.add(Dense(16, activation='relu'))
print( model.output_shape)
#Função de Ativação Softmax
model.add(Dense(1, activation='sigmoid'))
print( model.output_shape)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, validation_split = 0.2, epochs=20, batch_size=500)

In [ ]:
plt.figure(1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()